In [1]:
if not 'xl_settings_file_path' in locals():
    xl_settings_file_path = r'C:\GitHub\FUSE\Input\settings\cwatm_settings.xlsx'
if not 'output_folder' in locals():
    output_folder = r'P:\watmodel\CWATM\Saudi_Arabia\Outputs\Land'
import pandas as pd
if not 'withCrops' in locals():
    withCrops=False
if not 'withYield' in locals():
    withYield=False

In [2]:
def crops(xl_settings_file_path):
    import pandas as pd
    df = pd.read_excel(xl_settings_file_path, sheet_name='Crops')
    
    Crops = []
    for i in df.index:
        crop = [df['Planting month'][i]]
        
        for gs in range(1,5):
            gs_parameters = [df['GS'+str(gs)][i], df['KC'+str(gs)][i], df['KY'+str(gs)][i]]
            crop.append(gs_parameters)
        
        Crops.append(crop)
        
def crops_initialise(xl_settings_file_path):
    df = pd.read_excel(xl_settings_file_path, sheet_name='Crops')

    Crops = []
    Crops_names = []
    for i in df.index:
        crop = [df['Planting month'][i]]

        for gs in range(1, 5):
            gs_parameters = [df['GS' + str(gs)][i], df['KC' + str(gs)][i], df['KY' + str(gs)][i]]
            crop.append(gs_parameters)

        Crops.append(crop)
        Crops_names.append(df['Crop'][i])

    return Crops, Crops_names


In [3]:
import time
import pandas as pd
from datetime import date
from netCDF4 import Dataset, num2date
import numpy.ma as ma
import numpy as np



if withCrops:
    Crops, Crops_names = crops_initialise(xl_settings_file_path)
else:
    Crops = []
    Crops_names = []
    
# Yield_(non)Irr[c] 
if withYield:
    #Vars_Yield = [['Yield_Irr['+str(c)+']',   'Yield (Irr) '+Crops_names[c],   '%',  'Yield Irr'] for c in range(len(Crops))]
    #Vars_YieldnonIrr = [['Yield_nonIrr['+str(c)+']',    'Yield (non-Irr) '+Crops_names[c], '%','Yield non-Irr'] for c in range(len(Crops))]
    Vars_Yield = [['Yield_Irr['+str(c)+']',   'Yield (Irr) '+Crops_names[c],   '%',  'Yield Irr'] for c in [0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16]]
    Vars_YieldnonIrr = [['Yield_nonIrr['+str(c)+']',    'Yield (non-Irr) '+Crops_names[c], '%','Yield non-Irr'] for c in [0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16]]

else:
    Vars_Yield=[]
    Vars_YieldnonIrr=[]
    
# fracCrops_(non)Irr[c]
Vars = [['fracCrops_Irr['+str(c)+']',   Crops_names[c],   '%',  'Irrigated Area'] for c in range(len(Crops))]
Vars_nonIrr = [['fracCrops_nonIrr['+str(c)+']',   Crops_names[c], '%','non-Irrigated Area'] for c in range(len(Crops))]

# fracCrops_(non)IrrLandDemand[c]
Vars_proposed_Irr = [['fracCrops_IrrLandDemand['+str(c)+']',   Crops_names[c], '%','Irrigated Area proposed'] for c in range(len(Crops))]
Vars_proposed_nonIrr = [['fracCrops_nonIrrLandDemand['+str(c)+']',   Crops_names[c], '%','non-Irrigated Area proposed'] for c in range(len(Crops))]

# frac_total(non)Irr
Vars_frac_totalIrr = ['frac_totalIrr',   "Total irrigated", '%','Total irrigated and planted']
Vars_frac_totalnonIrr = ['frac_totalnonIrr',   "Total non-irrigated", '%','Total non-irrigated and planted']

# fracVegCover[0...5]
Vars_frac_Forest = ['fracVegCover[0]', 'Forest', '%', 'Forest']
Vars_frac_totalnonIrrLand = ['fracVegCover[1]',   "Total non-Irrigated", '%','Total non-Irrigated land']
Vars_frac_Paddy = ['fracVegCover[2]', 'Paddy', '%', 'Paddy']
Vars_frac_totalIrrLand = ['fracVegCover[3]',   "Total irrigated", '%','Total Irrigated land']
Vars_frac_Sealed = ['fracVegCover[4]', 'Sealed', '%', 'Sealed']
Vars_frac_Water = ['fracVegCover[5]', 'Water', '%', 'Water']
Vars_frac_Available = ['availableArableLand', 'Available land', '%', 'Available land']

# fallow(non)Irr
Vars_frac_fallowIrr = ['fallowIrr', 'fallowed irrigated land', '%', 'Fallowed Irrigated land']
Vars_frac_fallownonIrr = ['fallownonIrr', 'fallowed non-irrigated land', '%', 'Fallowed non-irrigated land']

# GeneralCrop_(non)Irr
Vars_frac_GeneralIrr = ['GeneralCrop_Irr', 'General irrigated land', '%', 'General irrigated land']
Vars_frac_GeneralnonIrr = ['GeneralCrop_nonIrr', 'General non-irrigated land', '%', 'General non-irrigated land']

Vars.extend(Vars_Yield)
Vars.extend(Vars_YieldnonIrr)
Vars.extend(Vars_nonIrr)
Vars.extend(Vars_proposed_Irr )
Vars.extend(Vars_proposed_nonIrr)
Vars.append(Vars_frac_totalIrr)
Vars.append(Vars_frac_totalnonIrr)
Vars.append(Vars_frac_totalIrrLand)
Vars.append(Vars_frac_totalnonIrrLand)
Vars.append(Vars_frac_fallowIrr)
Vars.append(Vars_frac_fallownonIrr)
Vars.append(Vars_frac_GeneralIrr)
Vars.append(Vars_frac_GeneralnonIrr)
Vars.append(Vars_frac_Forest)
Vars.append(Vars_frac_Paddy)
Vars.append(Vars_frac_Sealed)
Vars.append(Vars_frac_Water)
Vars.append(Vars_frac_Available)


#Vars.append()       ['fracCrops_nonIrr['+str(c)+']',   Crops_names[c],       '%',      'non-Irrigated Area']
keys = [i[-1] for i in Vars]
WB = {key: [] for key in keys} # Creates an empty set relating to each key, overriding the previous to leave only one of each 
#print('keys: ', keys)
cellArea = Dataset(output_folder +'/'+ 'cellArea_totalend.nc', 'r').variables['cellArea_totalend'][:,:]
cellAreaC = ma.compressed(cellArea)

#nc_simulated = Dataset(output_folder + '/fracCrops_Irr[1]_monthend.nc', 'r') #Could this be 0?
nc_simulated = Dataset(output_folder + '/fracVegCover[0]_monthend.nc', 'r')
Dates_simulation = num2date(nc_simulated.variables['time'][:], units=nc_simulated.variables['time'].units)


for t in range(len(Dates_simulation)):
    Dates_simulation[t] = np.datetime64(Dates_simulation[t])


for var in Vars:
    
    #print(var[0]) # Troubleshooting
    
    nc_filename = output_folder +'/'+ var[0] + '_monthend.nc'
    temp = Dataset(nc_filename, 'r').variables[var[0]+ '_monthend'][:,:,:]
    Daily_basinSum = []
    
    for d in range(len(Dates_simulation)):
        
        daily_basinSum = np.sum(np.multiply(ma.compressed(temp[d,:,:]), cellAreaC))
        Daily_basinSum.append(daily_basinSum)
    
    WB[var[-1]].append([var[1], np.array(Daily_basinSum)])
    #print([var[1], np.sum(np.array(Daily_basinSum))])


Vars2 = [['actTransTotal_grasslands','Transpiration (Grasslands)',   'M',    'actTransTotal_grasslands']]
Vars2.append(['actTransTotal_forest',     'Transpiration (Forest)',        'M',    'actTransTotal_forest']) 
Vars2.append(['actTransTotal_paddy',     'Transpiration (Paddy)',        'M',    'actTransTotal_paddy']) 
Vars2.append(['actTransTotal_nonpaddy',  'Transpiration (non-Paddy)',    'M',    'actTransTotal_nonpaddy'])
Vars2.append(['act_irrWithdrawal',     'Irrigation withdrawals',        'M',    'Irrigation withdrawals']) 
Vars2.append(['act_nonIrrWithdrawal',  'non-Irrigation withdrawals',    'M',    'non-Irrigation withdrawals'])
Vars2.append(['act_domWithdrawal',   'Domestic Withdrawal',          'M',  'Domestic withdrawals'])
Vars2.append(['act_indWithdrawal',   'Industrial Withdrawal',          'M',  'Industrial withdrawal'])
Vars2.append(['Rain',  'Rain',    'M',    'Rain'])
Vars2.append(['Snow',  'Snow',    'M',    'Snow'])

Vars2.append(['nonIrrDemand',  'non-Irrigation demand',    'M',    'non-Irrigation demand'])
Vars2.append(['totalIrrDemand',  'Irrigation demand',    'M',    'Irrigation demand'])
Vars2.append(['domesticDemand',  'domesticDemand',    'M',    'domesticDemand'])
Vars2.append(['industryDemand',  'industryDemand',    'M',    'industryDemand'])
Vars2.append(['livestockDemand',  'livestockDemand',    'M',    'livestockDemand'])
Vars2.append(['unmetDemand',  'unmetDemand',    'M',    'unmetDemand'])
Vars2.append(['act_irrPaddyDemand',  'act_irrPaddyDemand',    'M',    'act_irrPaddyDemand'])
Vars2.append(['act_irrNonpaddyDemand',  'act_irrNonpaddyDemand',    'M',    'act_irrNonpaddyDemand'])
Vars2.append(['Res_Domestic',  'Res_Domestic',    'M',    'Res_Domestic'])
Vars2.append(['GW_Domestic',  'GW_Domestic',    'M',    'GW_Domestic'])



keys2 = [i[-1] for i in Vars2]
WB2 = {key: [] for key in keys2}

# actTransTotal_grasslands, actTransTotal_paddy, actTransTotal_nonpaddy, act_irrWithdrawal act_nonIrrWithdrawal, cellArea_totalend

#Experiment
nc_simulated2 = Dataset(output_folder + '/actTransTotal_grasslands_daily.nc', 'r')
Dates_simulation2 = num2date(nc_simulated2.variables['time'][:], units=nc_simulated2.variables['time'].units)
#print('days: ', len(Dates_simulation2))
#print('Outptus are in m3 over the simulation period')
LULC_table = {'Outputs':[], 'Billion m3':[]}
for var in Vars2:
    
    #print(var[0])  #Troubleshooting
    
    nc_filename = output_folder +'/'+ var[0] + '_daily.nc'
    temp = Dataset(nc_filename, 'r').variables[var[0]][:,:,:]
    Daily_basinSum = []
    
    for d in range(len(Dates_simulation2)):
        
        daily_basinSum = np.sum(np.multiply(ma.compressed(temp[d,:,:]), cellAreaC))
        Daily_basinSum.append(daily_basinSum)

    #print([var[1], np.sum(np.array(Daily_basinSum))])
    if 'Demand' not in var[0]:
        LULC_table['Outputs'].append(var[1])
        LULC_table['Billion m3'].append(np.sum(np.array(Daily_basinSum))/1_000_000_000)
    
    WB2[var[-1]].append([var[1], np.array(Daily_basinSum)])

#data_WB = {'Water flows, total':['Inputs', 'Outputs', 'Storage Change', 'Balance', 'Balance over all flows (%):'],'BMC':[  total_input_adjusted/1_000_000_000,   total_output/1_000_000_000,   total_change/1_000_000_000,   balance/1_000_000_000, percent_accurate_WB]}
# Create DataFrame
df = pd.DataFrame(LULC_table)
df=df.style.set_table_styles([{'selector': 'th', 'props': [('font-size', '10pt')]}]).set_properties(**{"font-size": "10pt"}).hide(axis='index')
display(df)


FileNotFoundError: [Errno 2] No such file or directory: b'P:\\watmodel\\CWATM\\Saudi_Arabia\\Outputs\\Land/livestockDemand_daily.nc'

In [4]:
from plotly import graph_objs as go, offline as po, tools
fig_planted_Irr = go.Figure()
fig_planted_nonIrr = go.Figure()
fig_planted_total = go.Figure()

fig_daily = go.Figure()


for c in range(len(Crops)):
    fig_planted_Irr.add_trace(go.Scatter(y=WB['Irrigated Area'][c][1],
                             x=Dates_simulation,
                    mode='lines',
                    name=Crops_names[c]))
    
    fig_planted_nonIrr.add_trace(go.Scatter(y=WB['non-Irrigated Area'][c][1],
                         x=Dates_simulation,
                mode='lines',
                name=Crops_names[c]))
    
    fig_planted_total.add_trace(go.Scatter(y=WB['non-Irrigated Area'][c][1]+WB['Irrigated Area'][c][1],
                     x=Dates_simulation,
            mode='lines',
            name=Crops_names[c]))
    
    
    
    
    fig_planted_Irr.add_trace(go.Scatter(y=WB['Irrigated Area proposed'][c][1],
                         x=Dates_simulation,
                mode='lines',
                name='proposed'+Crops_names[c]))
    
    fig_planted_nonIrr.add_trace(go.Scatter(y=WB['non-Irrigated Area proposed'][c][1],
                         x=Dates_simulation,
                mode='lines',
                name='proposed'+Crops_names[c]))
    
    fig_planted_total.add_trace(go.Scatter(y=WB['non-Irrigated Area proposed'][c][1]+WB['Irrigated Area proposed'][c][1],
                     x=Dates_simulation,
            mode='lines',
            name='proposed'+ Crops_names[c]))
    
    
fig_planted_Irr.add_trace(go.Scatter(y=WB['Total Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total irrigated land'))

fig_planted_nonIrr.add_trace(go.Scatter(y=WB['Total non-Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total non-irrigated land'))   

fig_planted_total.add_trace(go.Scatter(y=WB['Total non-Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total non-irrigated land'))
fig_planted_total.add_trace(go.Scatter(y=WB['Total Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total irrigated land'))
    

fig_planted_Irr.add_trace(go.Scatter(y=WB['Total irrigated and planted'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total irrigated crops'))

fig_planted_nonIrr.add_trace(go.Scatter(y=WB['Total non-irrigated and planted'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total non-irrigated crops'))


fig_planted_total.add_trace(go.Scatter(y=WB['Total irrigated and planted'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total irrigated crops'))

fig_planted_total.add_trace(go.Scatter(y=WB['Total non-irrigated and planted'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='total non-irrigated crops'))

#dev
fig_planted_total.add_trace(go.Scatter(y=WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='fallowed irrigated crops'))

fig_planted_total.add_trace(go.Scatter(y=WB['Fallowed non-irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='fallowed non-irrigated crops'))


fig_planted_total.add_trace(go.Scatter(y=WB['General irrigated land'][0][1], #WB['Total Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='General irrigated crops'))

fig_planted_total.add_trace(go.Scatter(y=WB['General non-irrigated land'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='General non-irrigated crops'))





fig_planted_total.add_trace(go.Scatter(y=WB['Forest'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='Forest'))

fig_planted_total.add_trace(go.Scatter(y=WB['Paddy'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='Paddy'))

fig_planted_total.add_trace(go.Scatter(y=WB['Sealed'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='Sealed'))

fig_planted_total.add_trace(go.Scatter(y=WB['Water'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='Water'))
fig_planted_total.add_trace(go.Scatter(y=WB['Available land'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation,
    mode='lines',
    name='Available land'))

fig_daily.add_trace(go.Scatter(y=WB2['actTransTotal_forest'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Forest transpiration'))
fig_daily.add_trace(go.Scatter(y=WB2['actTransTotal_grasslands'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Grasslands transpiration'))
fig_daily.add_trace(go.Scatter(y=WB2['actTransTotal_paddy'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Paddy transpiration'))
fig_daily.add_trace(go.Scatter(y=WB2['actTransTotal_nonpaddy'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='non-Paddy transpiration'))
              
fig_daily.add_trace(go.Scatter(y=WB2['Irrigation demand'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Irrigation demand'))
fig_daily.add_trace(go.Scatter(y=WB2['non-Irrigation demand'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='non-Irrigation demand'))
fig_daily.add_trace(go.Scatter(y=WB2['domesticDemand'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='domesticDemand'))
fig_daily.add_trace(go.Scatter(y=WB2['industryDemand'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='industryDemand'))

fig_daily.add_trace(go.Scatter(y=WB2['Rain'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Rain'))
fig_daily.add_trace(go.Scatter(y=WB2['Snow'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Snow'))
fig_daily.add_trace(go.Scatter(y=WB2['act_irrPaddyDemand'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='act_irrPaddyDemand'))
fig_daily.add_trace(go.Scatter(y=WB2['act_irrNonpaddyDemand'][0][1], #y=WB['Total non-Irrigated land'][0][1] - WB['Total irrigated and planted'][0][1] - WB['Fallowed Irrigated land'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='act_irrNonpaddyDemand '))

fig_daily.add_trace(go.Scatter(y=WB2['Irrigation withdrawals'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='act_irrWithdrawal'))
fig_daily.add_trace(go.Scatter(y=WB2['non-Irrigation withdrawals'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='act_nonIrrWithdrawal'))
fig_daily.add_trace(go.Scatter(y=WB2['Domestic withdrawals'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='act_domWithdrawal'))
fig_daily.add_trace(go.Scatter(y=WB2['Industrial withdrawal'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='act_indWithdrawal'))
fig_daily.add_trace(go.Scatter(y=WB2['Res_Domestic'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='Res_Domestic'))
fig_daily.add_trace(go.Scatter(y=WB2['GW_Domestic'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='GW_Domestic'))

fig_daily.add_trace(go.Scatter(y=WB2['unmetDemand'][0][1],
             x=Dates_simulation2,
    mode='lines',
    name='unmetDemand'))

fig_daily.update_layout(title='Water use and request',
                       xaxis_title = 'Days',
                       yaxis_title = 'Volume (m3)',
                        template='plotly_dark')


"""
fig_planted_Irr.update_layout(title='Crops irrigated and planted',
                       xaxis_title = 'Months',
                       yaxis_title = 'Area (m2)')

fig_planted_Irr.update_layout(title='Crops non-irrigated and planted',
                       xaxis_title = 'Months',
                       yaxis_title = 'Area (m2)')

fig_planted_Irr.update_layout(title='Crops total planted',
                       xaxis_title = 'Months',
                       yaxis_title = 'Area (m2)')

"""
fig_planted_Irr.update_layout(title='Crops irrigated and planted',
                       xaxis_title = 'Months',
                       yaxis_title = 'Area (m2)',  template='plotly_dark')

fig_planted_nonIrr.update_layout(title='Crops non-irrigated and planted',
                       xaxis_title = 'Months',
                       yaxis_title = 'Area (m2)',  template='plotly_dark')

fig_planted_total.update_layout(title='Crops total planted',
                       xaxis_title = 'Months',
                       yaxis_title = 'Area (m2)', template='plotly_dark')

fig_planted_Irr.show()
fig_planted_nonIrr.show()
fig_planted_total.show()
fig_daily.show()

IndexError: list index out of range